In [77]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import os

In [83]:

#Model - this makes the network (ie the policy)

#we have the soft Q function, the parameterised value function V, and the parameterised tractable policy pi

# TODOs
# Make comments properly

# Whitening of inputs

# Need batching

# Need recall

# Train, test routines


# Here is a helper class to make a simple neural network. Importantly, it allows us to easily get the parameters, and hopefully to link the inputs to other variables
# The get_output functionality is borrowed from the SAC reference code.

class MLP():
    def __init__(self,name,inputs,output_size,n_hidden,n_layers):
        self._name = name
        self.inputs = inputs
        self.output_size = output_size
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        
        self.output = self.make_network(reuse = False)
        
    def make_network(self,inputs = False,reuse = tf.AUTO_REUSE):
        # This function just makes a simple fully connected network. It is structured in a little bit of a silly way. The idea is that this lets one reuse the network weights elsewhere with different inputs. Currently not actually using this functionality 
        if inputs is False :
            inputs = self.inputs
            
        with tf.variable_scope(self._name,reuse = reuse):
            if not(isinstance(inputs,tf.Tensor)):  # Can chuck in more than one input. This just concatenates them
                inputs = tf.concat(inputs,axis=1)

            # To do: understand weight initialization!   
            self.hidden = slim.stack(inputs, slim.fully_connected, [self.n_hidden]*self.n_layers, scope='fc',activation_fn=tf.nn.relu)
            outputs = slim.fully_connected(self.hidden,self.output_size,activation_fn=None)
        return outputs

    def get_params_internal(self):
        # Useful function to get network weights
        
        scope = tf.get_variable_scope().name
        scope += '/' + self._name + '/' if len(scope) else self._name + '/'

        return tf.get_collection(
            tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope
        )

class Qnet(MLP):
    # Make a simple q network
    def __init__(self,actions,obs,n_hidden,n_layers):
        # Ok so Q function takes s,a, gives Q   
        self.actions = actions
        self.obs = obs
        
        # Super is used to call the init method of the parent class
        super(Qnet,self).__init__('qNet',(self.actions,self.obs),1,n_hidden,n_layers)

class Vnet(MLP):
    # Make a simple v network
    def __init__(self,obs,n_hidden,n_layers):
        # Ok so V function takes s, gives V
        self.obs = obs
        super(Vnet,self).__init__('vNet',(self.obs),1,n_hidden,n_layers)

class Policy_Discrete(MLP):
    # Make a policy.
    # For now assume that discrete action space, such that tractable - obviously this slightly defeats the point of SAC implementation since all about how hard to compute the partition function
    # P function takes s, needs to be able to give actions.

    def __init__(self,action_size,obs,n_hidden,n_layers):

        self.obs = obs
        self.action_size = action_size
        self.discrete = True
        
        super(Policy_Discrete,self).__init__('policy',(self.obs),action_size,n_hidden,n_layers)
        self.make_policy_outputs(reuse=False)
        
    def make_policy_outputs(self, reuse = tf.AUTO_REUSE):
       
        with tf.variable_scope(self._name + '_outs',reuse = reuse):
            self.policy_output = tf.nn.softmax(self.output) # Automatically sum to one.
            self.log_policy_output = tf.log(self.policy_output)
            self.action = tf.multinomial(self.log_policy_output, num_samples=1)[0] # Will generate an action


    def get_action(self,obs):
        return self.action.eval(feed_dict = {self.obs : [obs]})[0]

class Soft_Actor_Critic():
    # This class handles the training of the networks
    def __init__(self,Qnet,Vnet,Policy,actions,obs,next_obs,rewards,dones,lr=3e-4,discount = 0.99, tau=0.01):
        self.lr = lr
        self.discount = discount
        self.tau = tau 
        
        # Maybe would be nicer to not pass these but define here, but this seems to be messy. Once check if works, could go back to defining here
        self.actions = actions
        self.obs = obs
        self.next_obs = next_obs
        self.rewards = rewards
        self.dones = dones
        
        self.Qnet = Qnet
        self.Vnet = Vnet
        self.Policy = Policy
        
        self.Qs = self.Qnet.output
        self.Vs = self.Vnet.output
        self.policy_log_a = self.Policy.log_policy_output
        
        # Duplicate v network for target.
        with tf.variable_scope('vNet_T'):
            self.target_Vs = self.Vnet.make_network(inputs = self.next_obs,reuse=False) # We dont reuse weights
            self.target_V_params = self.Vnet.get_params_internal()
            
        self.optimizer = tf.train.AdamOptimizer(learning_rate = self.lr)
        
        self.train_ops = []
        self.init_Q_net_training()
        self.init_V_net_training()
        self.init_Policy_training()
        self.init_target_v_update()
        
        
        
    def init_Q_net_training(self):
        training_variables = self.Qnet.get_params_internal()
        with tf.variable_scope('Q_loss'):
            Q_t = tf.stop_gradient(self.rewards + self.discount * (1-self.dones) * self.target_Vs)
            self.Q_Loss = 0.5*tf.reduce_mean(tf.square(self.Qs - Q_t))
            tf.summary.scalar('Q_loss', self.Q_Loss)
            
        self.train_Q = self.optimizer.minimize(self.Q_Loss,var_list = training_variables)
        self.train_ops.append(self.train_Q)
        
    def init_V_net_training(self):
        training_variables = self.Vnet.get_params_internal()

        with tf.variable_scope('V_loss'):
            V_t = tf.stop_gradient(self.Qs - self.policy_log_a) 
            self.V_Loss = 0.5*tf.reduce_mean(tf.square(self.Vs - V_t))
            tf.summary.scalar('V_loss', self.V_Loss)
            
        self.train_V = self.optimizer.minimize(self.V_Loss,var_list = training_variables)
        self.train_ops.append(self.train_V)

    def init_Policy_training(self):
        training_variables = self.Policy.get_params_internal()

        with tf.variable_scope('P_loss'):
            P_t = tf.stop_gradient(self.Qs - self.Vs) 
            self.P_Loss = 0.5*tf.reduce_mean(tf.square(self.policy_log_a - P_t))
            tf.summary.scalar('P_loss', self.P_Loss)
            
        self.train_P = self.optimizer.minimize(self.P_Loss,var_list = training_variables)
        self.train_ops.append(self.train_P)
        
    def init_target_v_update(self):
        # Pull the vnet params
        vnet_params = self.Vnet.get_params_internal()
        
        with tf.variable_scope('Vt_loss'):
            self.tvnet_update = []
            for tv_p in self.target_V_params:
                # Match each target net param with equiv from vnet
                v_p = [v for v in vnet_params if tv_p.name[(tv_p.name.index('/')+1):] in v.name]
                assert(len(v_p) == 1) # Check that only found one variable
                v_p = v_p[0]
                
                # Make sure that only runs once the training is done
                with tf.control_dependencies([self.train_V]):
                    self.tvnet_update.append(tv_p.assign(self.tau * v_p + (1-self.tau)*tv_p))
            self.tvnet_update = tf.group(self.tvnet_update)
            
        self.train_ops.append(self.tvnet_update)
        
    def _construct_feed_dict(self,samples):  
        return {self.actions : samples['actions'],
                    self.obs : samples['observations'],
                    self.next_obs : samples['next_observations'],
                    self.dones : samples['dones'],
                    self.rewards : samples['rewards']}
                    
    def train(self, samples, *args):
        feed_dict = self._construct_feed_dict(samples)
        return tf.get_default_session().run([self.train_ops] + list(args), feed_dict = feed_dict)[1:]

class replayBuffer():
    def __init__(self,n_inputs,n_outputs,max_buffer_size = 1e4):
        self._max_size = int(max_buffer_size)
        self.n_outputs = n_outputs
        self.n_inputs = n_inputs
        self.reset()
        
    def reset(self):
        self._size = 0
        self._pos = 0
        
        self.actions = np.zeros([self._max_size,self.n_outputs])
        self.observations = np.zeros([self._max_size,self.n_inputs])
        self.next_observations = np.zeros([self._max_size,self.n_inputs])
        self.rewards = np.zeros(self._max_size)
        self.dones = np.zeros(self._max_size)
        
    def add_sample(self,action,obs,next_obs,reward,done):
        self.actions[self._pos] = action
        self.observations[self._pos] = obs
        self.next_observations[self._pos] = next_obs
        self.rewards[self._pos] = reward
        self.dones[self._pos] = done
        
        self._advance()
    
    def _advance(self):
        self._pos = (self._pos + 1) % self._max_size
        if self._size < self._max_size:
            self._size += 1
            
    def get_samples(self,n_samples):
        inds = np.random.randint(0,self._size,n_samples)
        return dict(actions = self.actions[inds],
                   observations = self.observations[inds],
                   next_observations = self.next_observations[inds],
                   rewards = self.rewards[inds],
                   dones = self.dones[inds])
    
    def get_last_sample(self):
        last_pos = [(self._pos-1) % self._max_size]
        
        return dict(actions = self.actions[last_pos],
                   observations = self.observations[last_pos],
                   next_observations = self.next_observations[last_pos],
                   rewards = self.rewards[last_pos],
                   dones = self.dones[last_pos])
    
    def ready_to_sample(self,n_samples):
        return self._size >= n_samples
        
class logger():
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.vars = {}
        
    def record(self,var_name,value):
        if hasattr(self.vars,var_name):
            self.vars[var_name].append(value)
        else:
            self.vars[var_name] = [value]
    
    def get(self,var_name):
        if hasattr(self.vars,var_name):
            return self.vars[var_name]
        else:
            return False

class Sampler():
    def __init__(self,policy,env,replaybuffer):
        self.policy = policy
        self.env = env
        self.replaybuffer = replaybuffer
        
        self.reset()
        
    def reset(self): 
        self.current_obs = False
        self.episodes = 0
        self.current_episode_reward = 0
        self.mean_episode_reward = 0

    def sample(self):
        if (self.current_obs is False):
            self.current_obs = env.reset()
            
        action = pnet.get_action(self.current_obs)
        next_obs, reward, done, info = env.step(action)
        if self.policy.discrete == True:
            action = np.eye(self.policy.action_size)[action]
        rb.add_sample(action,self.current_obs,next_obs,reward,done)
        self.current_obs = next_obs
        
        self.current_episode_reward += reward

        if done:
            self.current_obs = False   
            self.episodes += 1

            log.record('episode_reward',self.current_episode_reward)
            self.mean_episode_reward = (self.mean_episode_reward * (self.episodes - 1) + self.current_episode_reward) / self.episodes
            self.current_episode_reward = 0
# action = tf.clip_by_value(logits,tf.expand_dims(env.action_space.low,0),tf.expand_dims(env.action_space.high,0)) # Have to clip the action space. This might be a bad idea


#should make so that the pi can be easily changed

#Algorithm ie Soft Actor Critic - training etc makes the ops

#Env

#optimizer


In [84]:
log_dir = os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                               'tensorflow/logs/soft_actor_critic')
env_name = 'CartPole-v0'
env = gym.make(env_name)

n_inputs = env.observation_space.shape[0]
n_outputs = env.action_space.n
    
epoch_length = 1000
max_epochs = 10
online_training = True
samples_per_env_step = 4
max_buffer_size = 1e3

n_hidden = 10
n_layers = 1
# Todo make these into lists so that can define each layer separately


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [85]:
tf.reset_default_graph() # THIS IS NECESSARY BEFORE MAKING NEW SESSION TO STOP IT ERRORING!!
try:
    sess
except:
    pass
else:
    sess.close()
    del sess
sess = tf.InteractiveSession()

rewards = tf.placeholder(tf.float32,shape = [None],name = 'rewards')
actions = tf.placeholder(tf.float32,shape = [None,n_outputs],name = 'actions')
observations = tf.placeholder(tf.float32,shape = [None,n_inputs],name = 'observations')
next_observations = tf.placeholder(tf.float32,shape = [None,n_inputs],name = 'next_observations')
dones = tf.placeholder(tf.float32,shape = [None],name = 'dones')

qnet = Qnet(actions,observations,n_hidden,n_layers)
vnet = Vnet(observations,n_hidden,n_layers)
pnet = Policy_Discrete(n_outputs,observations,n_hidden,n_layers)

sac = Soft_Actor_Critic(qnet,vnet,pnet,actions,observations,next_observations,rewards,dones)

rb = replayBuffer(n_inputs,n_outputs,max_buffer_size)
sampler = Sampler(pnet,env,rb)

            
log = logger() 
merged = tf.summary.merge_all()

writer = tf.summary.FileWriter(log_dir, sess.graph)

tf.global_variables_initializer().run()


In [86]:
for i in range(max_epochs):
    sampler.reset()
    
    for t in range(epoch_length):
        sampler.sample()
        
        if rb.ready_to_sample(samples_per_env_step):
            if online_training:
                samples = rb.get_last_sample()

            else:
                samples = rb.get_samples(samples_per_env_step)
            summary,losses = sac.train(samples,merged,[sac.P_Loss,sac.V_Loss,sac.Q_Loss])
        
    log.record('mean_episode_reward',sampler.mean_episode_reward)
    writer.add_summary(summary, i)
    print(losses)
    writer.flush()

    print('Epoch %i, mean_reward %d' % (i, sampler.mean_episode_reward))
    


[0.589618, 0.58961797, 0.10898283]
Epoch 0, mean_reward 22
[0.002862199, 0.002862199, 0.39234647]
Epoch 1, mean_reward 20
[0.017328588, 0.017328678, 2.0288508]
Epoch 2, mean_reward 19
[0.2266708, 0.22667105, 0.09445925]
Epoch 3, mean_reward 22
[0.054023813, 0.054023776, 0.5476971]
Epoch 4, mean_reward 21
[0.03840465, 0.03840478, 0.5763748]
Epoch 5, mean_reward 21
[0.35432103, 0.35432062, 1.0705687]
Epoch 6, mean_reward 22
[0.052163098, 0.052163254, 0.5816642]
Epoch 7, mean_reward 22
[0.030299729, 0.030299846, 0.834272]
Epoch 8, mean_reward 23
[0.032300506, 0.03230042, 0.7994556]
Epoch 9, mean_reward 22
